In [ ]:
from webdav3.client import Client
import os

# Bitte Login und Passwort eintragen

upload_dir= 'FOSInfo'

local_dir = 'Lernsax'

lernsax = {
    'webdav_hostname': 'https://www.lernsax.de/webdav.php',
    'webdav_login': '',
    'webdav_password': '',
    'disable_check': True
}

In [ ]:
# Download

private_dir = os.path.join(lernsax["webdav_login"],'storage')
full_upload_path = os.path.join(private_dir,upload_dir)

client = Client(lernsax)
private_dir_list = client.list(private_dir)

if f'{upload_dir}/' in private_dir_list:
    upload_dir_list = client.list(full_upload_path)
    
    if f'{local_dir}/' in upload_dir_list:
        remote_dir = os.path.join(full_upload_path,local_dir)
        
        download_paths = [(local_dir,remote_dir)]
        
        while len(download_paths) > 0:
            local_path, remote_path = download_paths.pop()
            
            dir_list = client.list(remote_path)[1:]
            print(dir_list)
            
            for item in dir_list:

                if item.endswith('/'):
                    remote_item_path = os.path.join(remote_path,item[:-1])
                    local_item_path = os.path.join(local_path,item[:-1])
                    os.makedirs(local_item_path,exist_ok=True)
                    download_paths.append((remote_item_path,remote_item_path))
                else:
                    try:
                        remote_item_path = os.path.join(remote_path,item)
                        local_item_path = os.path.join(local_path,item)
                        print(f'Download: {remote_item_path} -> {local_item_path}')
                        client.download(remote_item_path,local_item_path)
                    except:
                        print('Download Error')

In [ ]:
# Upload

private_dir = os.path.join(lernsax["webdav_login"],'storage')
full_upload_path = os.path.join(private_dir,upload_dir)

client = Client(lernsax)
private_dir_list = client.list(private_dir)

if f'{upload_dir}/'  not in private_dir_list:
    client.mkdir(full_upload_path)
    print('Create Directory')
    
for (dirpath, dirnames, filenames) in os.walk(local_dir):
    if os.path.basename(dirpath).startswith('.'):
        continue
    
    remote_path = os.path.join(full_upload_path,dirpath)
    client.mkdir(remote_path)
    
    for filename in filenames:
        if filename.startswith('.'):
            continue
        local_path = os.path.join(dirpath,filename)
        remote_path =  os.path.join(full_upload_path, local_path)
        
        print(f'Upload: {local_path} -> {remote_path}')
        client.upload(remote_path,local_path)